In [1]:
from train import train_per_epoch, get_data_loader
from torchvision import transforms
import torchvision
from PIL import Image
import segmentation_models_pytorch as smp
import utils.augment as T
import cv2
import numpy as np

In [2]:
transform = transforms.Compose([
        transforms.ToPILImage(),
        transforms.Resize((512, 1024),interpolation=Image.NEAREST),
])

In [3]:
net = smp.Unet(encoder_name="mobilenet_v2",classes= 19, encoder_weights='imagenet')

In [4]:
num_epochs = 50
max_acc = 0
patience = 10
not_improved_count = 0
batch_size = 4

In [12]:

train_loader = get_data_loader(datapth='datasets/cityscapes',annpath='datasets/cityscapes/train.txt',ims_per_gpu=batch_size,trans_func=transform,mode='train')
val_loader = get_data_loader(datapth='datasets/cityscapes',annpath='datasets/cityscapes/val.txt',ims_per_gpu=batch_size,trans_func=transform,mode='val')
# val_loader = DataLoader(val_dataset, batch_size=batch_size,
#                             shuffle=True, num_workers=6)

In [14]:
len(train_loader)

743

In [25]:
datapath = 'datasets/cityscapes'
list_folder = os.listdir(datapath)

In [57]:
def get_filepaths(directory):
    """
    This function will generate the file names in a directory 
    tree by walking the tree either top-down or bottom-up. For each 
    directory in the tree rooted at directory top (including top itself), 
    it yields a 3-tuple (dirpath, dirnames, filenames).
    """
    file_paths = []  # List which will store all of the full filepaths.

    # Walk the tree.
    for root, directories, files in os.walk(directory):
        for filename in files:
            # Join the two strings in order to form the full filepath.
            filepath = os.path.join(root, filename).replace('\\','/')
            file_paths.append(filepath)  # Add it to the list.

    return file_paths  # Self-explanatory.

# Run the above function and store its results in a variable.   
full_file_paths = get_filepaths("datasets/cityscapes/gtFine")

In [59]:
full_file_paths0 = get_filepaths("datasets/cityscapes/leftImg8bit")
full_file_paths1 = get_filepaths("datasets/cityscapes/gtFine")